In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization

In [3]:
# In the following, the fixed_constraints are constructed from the board directly. 
# This part only needs to be constructed once. The output has been returned as a sparse matrix for efficiency.

def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))

In [4]:
# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE

In [5]:
!pip install cvxopt

In [6]:
from cvxopt import solvers, matrix
import time
solvers.options['show_progress'] = False

In [7]:
def solver(data):
    corr_cnt = 0
    start = time.time()
    for i in range(len(data)):
        quiz = data["quizzes"][i]
        solu = data["solutions"][i]
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])


        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)

        sol = solvers.lp(c,G,h,H,b)
        
        # postprocessing the solution
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]
    
        # map to board
        z = np.reshape(x, (81, 9))
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                          - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
            pass
        else:
            #print("CORRECT")
            corr_cnt += 1
    
        if (i+1) % 5 == 0:
            end = time.time()
            print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
            
    end = time.time()
    print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
    return z

In [8]:
# data = pd.read_csv("small1.csv") 

In [9]:
# ans=solver(data)

Aver Time:   1.79 secs. Success rate: 5 / 5 
Aver Time:   1.84 secs. Success rate: 10 / 10 
Aver Time:   1.95 secs. Success rate: 15 / 15 
Aver Time:   1.93 secs. Success rate: 20 / 20 
Aver Time:   1.89 secs. Success rate: 24 / 24 


In [12]:
# data2 = pd.read_csv("small2.csv") 

In [22]:
# ans2 = solver(data2)

Aver Time:   2.11 secs. Success rate: 3 / 5 
Aver Time:   2.08 secs. Success rate: 6 / 10 
Aver Time:   2.03 secs. Success rate: 9 / 15 
Aver Time:   2.03 secs. Success rate: 13 / 20 
Aver Time:   2.04 secs. Success rate: 16 / 25 
Aver Time:   2.04 secs. Success rate: 19 / 30 
Aver Time:   2.16 secs. Success rate: 23 / 35 
Aver Time:   2.15 secs. Success rate: 25 / 40 
Aver Time:   2.13 secs. Success rate: 28 / 45 
Aver Time:   2.13 secs. Success rate: 31 / 50 
Aver Time:   2.12 secs. Success rate: 35 / 55 
Aver Time:   2.11 secs. Success rate: 39 / 60 
Aver Time:   2.12 secs. Success rate: 44 / 65 
Aver Time:   2.11 secs. Success rate: 48 / 70 
Aver Time:   2.12 secs. Success rate: 49 / 75 
Aver Time:   2.13 secs. Success rate: 52 / 80 
Aver Time:   2.17 secs. Success rate: 54 / 85 
Aver Time:   2.17 secs. Success rate: 56 / 90 
Aver Time:   2.16 secs. Success rate: 60 / 95 
Aver Time:   2.18 secs. Success rate: 63 / 100 
Aver Time:   2.18 secs. Success rate: 67 / 105 
Aver Time:   2.

Aver Time:   2.19 secs. Success rate: 304 / 850 
Aver Time:   2.19 secs. Success rate: 305 / 855 
Aver Time:   2.19 secs. Success rate: 306 / 860 
Aver Time:   2.19 secs. Success rate: 306 / 865 
Aver Time:   2.18 secs. Success rate: 307 / 870 
Aver Time:   2.18 secs. Success rate: 308 / 875 
Aver Time:   2.18 secs. Success rate: 309 / 880 
Aver Time:   2.18 secs. Success rate: 310 / 885 
Aver Time:   2.18 secs. Success rate: 311 / 890 
Aver Time:   2.17 secs. Success rate: 311 / 895 
Aver Time:   2.17 secs. Success rate: 312 / 900 
Aver Time:   2.17 secs. Success rate: 312 / 905 
Aver Time:   2.17 secs. Success rate: 312 / 910 
Aver Time:   2.16 secs. Success rate: 312 / 915 
Aver Time:   2.16 secs. Success rate: 315 / 920 
Aver Time:   2.16 secs. Success rate: 315 / 925 
Aver Time:   2.16 secs. Success rate: 316 / 930 
Aver Time:   2.16 secs. Success rate: 316 / 935 
Aver Time:   2.16 secs. Success rate: 318 / 940 
Aver Time:   2.16 secs. Success rate: 319 / 945 
Aver Time:   2.16 se

In [90]:
# data3 = pd.read_csv("large1.csv") 
# size1 = len(data3)

In [72]:
# import random
# random.seed(42)
# list1 = np.random.randint(0,size1,1000)
# list1 = list1.tolist()

In [95]:
# data31 = data3.loc[list1]
# data31.index = range(0,len(data31))

In [96]:
# ans3 = solver(data31)

Aver Time:   2.14 secs. Success rate: 5 / 5 
Aver Time:   2.11 secs. Success rate: 10 / 10 
Aver Time:   2.06 secs. Success rate: 14 / 15 
Aver Time:   2.07 secs. Success rate: 18 / 20 
Aver Time:   2.05 secs. Success rate: 21 / 25 
Aver Time:   2.05 secs. Success rate: 24 / 30 
Aver Time:   2.07 secs. Success rate: 28 / 35 
Aver Time:   2.07 secs. Success rate: 31 / 40 
Aver Time:   2.06 secs. Success rate: 34 / 45 
Aver Time:   2.05 secs. Success rate: 39 / 50 
Aver Time:   2.06 secs. Success rate: 43 / 55 
Aver Time:   2.05 secs. Success rate: 48 / 60 
Aver Time:   2.05 secs. Success rate: 52 / 65 
Aver Time:   2.05 secs. Success rate: 57 / 70 
Aver Time:   2.05 secs. Success rate: 61 / 75 
Aver Time:   2.05 secs. Success rate: 66 / 80 
Aver Time:   2.04 secs. Success rate: 70 / 85 
Aver Time:   2.04 secs. Success rate: 73 / 90 
Aver Time:   2.04 secs. Success rate: 78 / 95 
Aver Time:   2.04 secs. Success rate: 82 / 100 
Aver Time:   2.03 secs. Success rate: 87 / 105 
Aver Time:   

Aver Time:   2.00 secs. Success rate: 672 / 850 
Aver Time:   2.00 secs. Success rate: 677 / 855 
Aver Time:   2.00 secs. Success rate: 682 / 860 
Aver Time:   2.00 secs. Success rate: 686 / 865 
Aver Time:   2.00 secs. Success rate: 690 / 870 
Aver Time:   2.00 secs. Success rate: 695 / 875 
Aver Time:   2.00 secs. Success rate: 696 / 880 
Aver Time:   2.00 secs. Success rate: 700 / 885 
Aver Time:   2.00 secs. Success rate: 704 / 890 
Aver Time:   2.00 secs. Success rate: 709 / 895 
Aver Time:   2.00 secs. Success rate: 711 / 900 
Aver Time:   2.00 secs. Success rate: 715 / 905 
Aver Time:   2.00 secs. Success rate: 718 / 910 
Aver Time:   2.00 secs. Success rate: 723 / 915 
Aver Time:   2.00 secs. Success rate: 728 / 920 
Aver Time:   2.00 secs. Success rate: 731 / 925 
Aver Time:   2.00 secs. Success rate: 735 / 930 
Aver Time:   2.00 secs. Success rate: 740 / 935 
Aver Time:   2.00 secs. Success rate: 745 / 940 
Aver Time:   2.00 secs. Success rate: 749 / 945 
Aver Time:   2.00 se

In [100]:
# data4 = pd.read_csv("large2.csv")
# size2 = len(data4)
# random.seed(42)
# list2 = np.random.randint(0,size2,1000)
# list2 = list2.tolist()
# data41 = data4.loc[list2]
# data41.index = range(0,len(data41))

In [98]:
# ans4 = solver(data41)

Aver Time:   1.94 secs. Success rate: 5 / 5 
Aver Time:   1.93 secs. Success rate: 10 / 10 
Aver Time:   1.99 secs. Success rate: 15 / 15 
Aver Time:   1.96 secs. Success rate: 20 / 20 
Aver Time:   1.89 secs. Success rate: 25 / 25 
Aver Time:   1.84 secs. Success rate: 30 / 30 
Aver Time:   1.81 secs. Success rate: 35 / 35 
Aver Time:   1.78 secs. Success rate: 40 / 40 
Aver Time:   1.76 secs. Success rate: 45 / 45 
Aver Time:   1.74 secs. Success rate: 50 / 50 
Aver Time:   1.73 secs. Success rate: 55 / 55 
Aver Time:   1.72 secs. Success rate: 60 / 60 
Aver Time:   1.71 secs. Success rate: 65 / 65 
Aver Time:   1.70 secs. Success rate: 70 / 70 
Aver Time:   1.70 secs. Success rate: 75 / 75 
Aver Time:   1.69 secs. Success rate: 80 / 80 
Aver Time:   1.69 secs. Success rate: 85 / 85 
Aver Time:   1.68 secs. Success rate: 90 / 90 
Aver Time:   1.68 secs. Success rate: 95 / 95 
Aver Time:   1.67 secs. Success rate: 100 / 100 
Aver Time:   1.67 secs. Success rate: 105 / 105 
Aver Time: 

Aver Time:   1.62 secs. Success rate: 850 / 850 
Aver Time:   1.62 secs. Success rate: 855 / 855 
Aver Time:   1.62 secs. Success rate: 860 / 860 
Aver Time:   1.62 secs. Success rate: 865 / 865 
Aver Time:   1.62 secs. Success rate: 870 / 870 
Aver Time:   1.62 secs. Success rate: 875 / 875 
Aver Time:   1.62 secs. Success rate: 880 / 880 
Aver Time:   1.62 secs. Success rate: 885 / 885 
Aver Time:   1.62 secs. Success rate: 890 / 890 
Aver Time:   1.62 secs. Success rate: 895 / 895 
Aver Time:   1.62 secs. Success rate: 900 / 900 
Aver Time:   1.62 secs. Success rate: 905 / 905 
Aver Time:   1.62 secs. Success rate: 910 / 910 
Aver Time:   1.62 secs. Success rate: 915 / 915 
Aver Time:   1.62 secs. Success rate: 920 / 920 
Aver Time:   1.62 secs. Success rate: 925 / 925 
Aver Time:   1.62 secs. Success rate: 930 / 930 
Aver Time:   1.62 secs. Success rate: 935 / 935 
Aver Time:   1.62 secs. Success rate: 940 / 940 
Aver Time:   1.62 secs. Success rate: 945 / 945 
Aver Time:   1.62 se